# Agent LLM Renault : Analyse de Données Stratégiques et Financières

## Configuration Globale et Paramètres de Débogage
Ce notebook vise à développer un agent LLM pour répondre à des questions en utilisant des données structurées et non structurées sur Renault.

In [106]:
# --- Paramètres Globaux de Débogage et d'Exécution ---

# Mettez à True pour afficher les détails des chunks récupérés par l'outil RAG (knowledge_base_retriever)
# Utile pour analyser la pertinence des informations fournies au LLM.
DEBUG_SHOW_CHUNK_DETAILS = False

# Mettez à True pour afficher les "pensées" et actions internes de l'agent (logs verbeux)
# Utile pour comprendre le processus de décision de l'agent.
SHOW_AGENT_THOUGHTS = False

# Répertoire pour stocker les documents téléchargés (PDFs, transcriptions YouTube)
DOCS_PATH = "renault_documents"

# Chemin pour sauvegarder/charger l'index FAISS précalculé
FAISS_INDEX_PATH = "faiss_renault_index"

!pip install python-dotenv -q

# Token API Hugging Face (sera demandé si non trouvé dans les variables d'environnement)
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env (s'il existe)
load_dotenv()

# Maintenant, vous pouvez essayer de récupérer le token depuis l'environnement
HUGGINGFACEHUB_API_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
# HUGGINGFACEHUB_API_TOKEN = "hf_VOTRE_TOKEN_PERSONNEL_ICI"

## 1. Installation des Dépendances et Importations

In [100]:
# --- 1.1 Installation des bibliothèques nécessaires ---
print("⏳ Installation des bibliothèques...")
!pip install langchain langchain_community langchain_huggingface -q
z!pip install yfinance -q
!pip install pymupdf sentence-transformers faiss-cpu -q # PyMuPDF (fitz), SentenceTransformers, FAISS
!pip install matplotlib pandas scipy -q
!pip install transformers accelerate -q # Pour les modèles Hugging Face
!pip install requests youtube-transcript-api -q # Pour le téléchargement de données
print("✅ Installation des bibliothèques terminée.")

# --- 1.2 Importations principales ---
import os
import glob
import re
import json
import ast # Pour ast.literal_eval
from getpass import getpass

import pandas as pd
import numpy as np
import yfinance as yf
import fitz # PyMuPDF
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound
import requests

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import pearsonr

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
from langchain.agents import tool, AgentExecutor, create_react_agent
from langchain import hub

from IPython.display import Markdown, display, Image

print("📚 Importations principales effectuées.")

⏳ Installation des bibliothèques...
✅ Installation des bibliothèques terminée.
📚 Importations principales effectuées.


## 2. Acquisition des Données

In [102]:
# --- 2.1 Définition des sources de données ---
SOURCES = {
    "pdf": {
        "Renault_DEU_2020.pdf": "https://www.renaultgroup.com/Document-denregistrement-universel-2020/files/assets/common/downloads/files/RENAULT2020-URD-FR-ecobook.pdf",
        "Renault_DEU_2021.pdf": "https://www.bnains.org/archives/communiques/Renault/20220324_Document_d_enregistrement_universel_2021_Renault.pdf",
        "Renault_DEU_2022.pdf": "https://assets.renaultgroup.com/uploads/2024/12/renault_deu_fr-2022.pdf",
        "Renault_DEU_2023.pdf": "https://assets.renaultgroup.com/uploads/2024/10/renault_deu_2023_fr_01.pdf",
        "Renault_DEU_2024.pdf": "https://assets.renaultgroup.com/uploads/2025/04/Renault_DEU_2024.pdf",
    },
    "youtube": {
        "Plan_Renaulution_2021.txt": "https://www.youtube.com/watch?v=EtivAvmDr2Q",
        "Conference_Resultats_2021.txt": "https://www.youtube.com/watch?v=VfIeaIFSCQA", # FY 2021
        "Conference_Resultats_2022.txt": "https://www.youtube.com/watch?v=UWHlyjVtwT8", # FY 2022
        "Conference_Resultats_2023.txt": "https://www.youtube.com/watch?v=B57wephix-w", # FY 2023
        "Conference_Resultats_2024.txt": "https://www.youtube.com/watch?v=BA5ZOtWfpY0", # FY 2024
    }
}
os.makedirs(DOCS_PATH, exist_ok=True)
print(f"📁 Dossier pour les documents : '{DOCS_PATH}'")

# --- 2.2 Fonctions de téléchargement ---
def download_pdf(url: str, filename: str):
    """Télécharge un fichier PDF depuis une URL et le sauvegarde localement."""
    filepath = os.path.join(DOCS_PATH, filename)
    if os.path.exists(filepath):
        print(f"  ✅ Fichier PDF déjà présent : {filename}")
        return
    try:
        response = requests.get(url, stream=True, timeout=30)
        response.raise_for_status()
        with open(filepath, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"  📄 PDF téléchargé : {filename}")
    except requests.exceptions.RequestException as e:
        print(f"  ❌ Erreur de téléchargement (RequestException) pour {filename} depuis {url}: {e}")
    except Exception as e:
        print(f"  ❌ Erreur inattendue lors du téléchargement de {filename}: {e}")

def get_youtube_transcript(video_url: str, filename: str):
    """Extrait la transcription d'une vidéo YouTube et la sauvegarde en fichier texte."""
    filepath = os.path.join(DOCS_PATH, filename)
    if os.path.exists(filepath):
        print(f"  ✅ Transcription déjà présente : {filename}")
        return

    if not video_url or "URL_YOUTUBE" in video_url: # Vérifier si l'URL est un placeholder
        print(f"  ⚠️ URL YouTube non valide ou placeholder pour {filename}: {video_url}. Transcription ignorée.")
        return

    try:
        video_id = None
        if "v=" in video_url:
            video_id = video_url.split("v=")[1].split("&")[0]
        elif "youtu.be/" in video_url:
            video_id = video_url.split("youtu.be/")[1].split("?")[0]

        if not video_id:
            print(f"  ❌ Impossible d'extraire l'ID vidéo de l'URL {video_url} pour {filename}.")
            return

        transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=['fr', 'en'])
        transcript_text = " ".join([item['text'] for item in transcript_list])
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(transcript_text)
        print(f"  🎤 Transcription sauvegardée : {filename}")
    except NoTranscriptFound:
        print(f"  ❌ Aucune transcription trouvée (fr/en) pour la vidéo {video_url} ({filename}).")
    except Exception as e:
        print(f"  ❌ Erreur d'extraction de transcription pour {filename} ({video_url}): {e}")

# --- 2.3 Exécution du téléchargement ---
print("\n--- ⏳ Lancement de l'acquisition des documents ---")
for filename, url in SOURCES["pdf"].items():
    download_pdf(url, filename)

for filename, url in SOURCES["youtube"].items():
    get_youtube_transcript(url, filename)
print("--- ✅ Acquisition des documents terminée ---")

# --- 2.4 Données boursières ---
def fetch_stock_data(tickers: list, start_date: str, end_date: str) -> pd.DataFrame | None:
    """Télécharge les données boursières pour une liste de tickers."""
    print(f"\n⏳ Téléchargement des données boursières pour {tickers} de {start_date} à {end_date}...")
    try:
        data = yf.download(tickers, start=start_date, end=end_date, progress=False)
        if data.empty:
            print(f"  ⚠️ Aucune donnée retournée par yfinance pour {tickers} sur la période.")
            return None
        print(f"  📈 Données boursières téléchargées.")
        return data
    except Exception as e:
        print(f"  ❌ Erreur lors du téléchargement des données boursières : {e}")
        return None

current_date_str = pd.Timestamp.now().strftime('%Y-%m-%d')
stock_data = fetch_stock_data(
    tickers=['RNO.PA', '^FCHI'], # Ticker Renault et CAC40
    start_date="2020-01-01",
    end_date=current_date_str
)

if stock_data is not None and not stock_data.empty:
    print("\n Aperçu des données boursières (prix de clôture) :")
    print(stock_data['Close'].head())
else:
    print("\n⚠️ Aucune donnée boursière n'a été chargée. Les outils dépendant des actions pourraient ne pas fonctionner.")

📁 Dossier pour les documents : 'renault_documents'

--- ⏳ Lancement de l'acquisition des documents ---
  📄 PDF téléchargé : Renault_DEU_2020.pdf
  📄 PDF téléchargé : Renault_DEU_2021.pdf
  📄 PDF téléchargé : Renault_DEU_2022.pdf
  📄 PDF téléchargé : Renault_DEU_2023.pdf
  📄 PDF téléchargé : Renault_DEU_2024.pdf
  🎤 Transcription sauvegardée : Plan_Renaulution_2021.txt
  🎤 Transcription sauvegardée : Conference_Resultats_2021.txt
  🎤 Transcription sauvegardée : Conference_Resultats_2022.txt
  🎤 Transcription sauvegardée : Conference_Resultats_2023.txt
  🎤 Transcription sauvegardée : Conference_Resultats_2024.txt
--- ✅ Acquisition des documents terminée ---

⏳ Téléchargement des données boursières pour ['RNO.PA', '^FCHI'] de 2020-01-01 à 2025-06-01...
  📈 Données boursières téléchargées.

 Aperçu des données boursières (prix de clôture) :
Ticker         RNO.PA        ^FCHI
Date                              
2020-01-02  38.165432  6041.500000
2020-01-03  37.546040  6044.160156
2020-01-06 

## 3. Traitement des Documents et Création de la Base Vectorielle (RAG)



In [103]:
# --- 3.1 Initialisation du modèle d'embedding ---
# Ce modèle est utilisé pour vectoriser les documents et les requêtes.
print("⏳ Initialisation du modèle d'embedding...")
embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
print(f"✅ Modèle d'embedding '{embedding_model_name}' chargé.")

# --- 3.2 Chargement ou Création de l'Index FAISS ---
loaded_vector_store = None

if os.path.exists(FAISS_INDEX_PATH) and os.path.isdir(FAISS_INDEX_PATH):
    print(f"\n⏳ Tentative de chargement de l'index FAISS existant depuis '{FAISS_INDEX_PATH}'...")
    try:
        loaded_vector_store = FAISS.load_local(
            FAISS_INDEX_PATH,
            embeddings_model,
            allow_dangerous_deserialization=True
        )
        print("✅ Index FAISS existant chargé avec succès.")
    except Exception as e:
        print(f"❌ Erreur lors du chargement de l'index FAISS existant: {e}. L'index sera recréé.")
        loaded_vector_store = None
else:
    print(f"\nℹ️ Aucun index FAISS existant trouvé à '{FAISS_INDEX_PATH}'. L'index sera créé.")

if loaded_vector_store is None:
    print("\n⏳ Création d'un nouvel index FAISS...")

    print("  Étape 1/4 : Chargement des documents...")
    all_docs = []
    if not os.path.exists(DOCS_PATH):
        print(f"   ❌ ERREUR: Dossier des documents '{DOCS_PATH}' introuvable. Impossible de créer l'index.")
    else:
        doc_path_pattern = os.path.join(DOCS_PATH, "*")
        file_paths = glob.glob(doc_path_pattern)
        if not file_paths:
            print(f"   ⚠️ Aucun fichier trouvé dans '{DOCS_PATH}'.")

        for file_path in file_paths:
            try:
                filename = os.path.basename(file_path)
                if file_path.lower().endswith(".pdf"):
                    print(f"      -> Chargement PDF : {filename}")
                    loader = PyMuPDFLoader(file_path)
                    docs = loader.load()
                elif file_path.lower().endswith(".txt"):
                    print(f"      -> Chargement TXT : {filename}")
                    loader = TextLoader(file_path, encoding='utf-8')
                    docs = loader.load()
                else:
                    docs = [] # Ignorer autres types de fichiers

                # Ajouter le nom du fichier comme métadonnée principale si manquant
                for doc in docs:
                    if 'source' not in doc.metadata:
                        doc.metadata['source'] = file_path
                    doc.metadata['filename'] = filename # Pour un accès plus simple
                all_docs.extend(docs)
            except Exception as e:
                print(f"      ⚠️ Erreur lors du chargement de {filename}: {e}")

    if not all_docs:
        print("\n   ❌ Aucun document n'a pu être chargé. L'index FAISS ne sera pas créé.")
    else:
        print(f"\n   ✅ {len(all_docs)} pages/documents chargés.")

        print("\n  Étape 2/4 : Découpage des documents en 'chunks'...")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
            add_start_index=True
        )
        chunks = text_splitter.split_documents(all_docs)
        print(f"   ✅ Documents découpés en {len(chunks)} 'chunks'.")

        if not chunks:
            print("   ❌ Aucun 'chunk' n'a été créé. Impossible de construire l'index FAISS.")
        else:
            print("\n  Étape 3/4 : Création des embeddings et de la base vectorielle FAISS...")
            loaded_vector_store = FAISS.from_documents(chunks, embeddings_model)
            print("   ✅ Base vectorielle créée.")

            print(f"\n  Étape 4/4 : Sauvegarde de la base vectorielle dans '{FAISS_INDEX_PATH}'...")
            try:
                loaded_vector_store.save_local(FAISS_INDEX_PATH)
                print(f"   💾 Base vectorielle sauvegardée.")
            except Exception as e:
                print(f"   ❌ Erreur lors de la sauvegarde de l'index FAISS : {e}")

# --- 3.3 Test Optionnel de la Base Vectorielle ---
if loaded_vector_store:
    print("\n⏳ Test de la base vectorielle (chargée ou nouvellement créée)...")
    try:
        test_query = "Qu'est-ce que le plan Renaulution ?"
        test_results = loaded_vector_store.similarity_search(test_query, k=2)
        print(f"\n  🔍 Résultats de recherche pour '{test_query}':")
        if test_results:
            for i, doc in enumerate(test_results):
                print(f"    --- Résultat {i+1} ---")
                print(f"    Source: {doc.metadata.get('filename', doc.metadata.get('source', 'N/A'))}")
                # print(f"    Contenu: {doc.page_content[:300]}...")
        else:
            print("    Aucun résultat trouvé.")
    except Exception as e:
        print(f"  ❌ Erreur lors du test de la base vectorielle : {e}")
else:
    print("\n❌ Base vectorielle non disponible. Le RAG ne fonctionnera pas.")

⏳ Initialisation du modèle d'embedding...
✅ Modèle d'embedding 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' chargé.

ℹ️ Aucun index FAISS existant trouvé à 'faiss_renault_index'. L'index sera créé.

⏳ Création d'un nouvel index FAISS...
  Étape 1/4 : Chargement des documents...
      -> Chargement TXT : Conference_Resultats_2024.txt
      -> Chargement TXT : Conference_Resultats_2023.txt
      -> Chargement PDF : Renault_DEU_2020.pdf
      -> Chargement PDF : Renault_DEU_2022.pdf
      -> Chargement TXT : Conference_Resultats_2022.txt
      -> Chargement PDF : Renault_DEU_2023.pdf
      -> Chargement TXT : Plan_Renaulution_2021.txt
      -> Chargement TXT : Conference_Resultats_2021.txt
      -> Chargement PDF : Renault_DEU_2021.pdf
      -> Chargement PDF : Renault_DEU_2024.pdf

   ✅ 2780 pages/documents chargés.

  Étape 2/4 : Découpage des documents en 'chunks'...
   ✅ Documents découpés en 15081 'chunks'.

  Étape 3/4 : Création des embeddings et de la base vectori

## 4. Création des Outils

In [104]:
# --- Outils pour l'Agent ---
@tool
def knowledge_base_retriever(tool_input: str) -> str:
    """
    Interroge la base de connaissances (documents Renault) pour trouver des informations pertinentes.
    Retourne les extraits les plus significatifs.
    Pour cibler une année, l'Action Input peut être une chaîne JSON contenant les clés "query" et "target_year".
    Exemple pour une année spécifique: '{"query": "membres conseil administration 2021", "target_year": "2021"}'
    Exemple pour une requête générale: "plan Renaulution objectifs"
    """
    global DEBUG_SHOW_CHUNK_DETAILS
    print(f"🛠️ Outil 'knowledge_base_retriever' appelé avec tool_input: {repr(tool_input)}")

    query_str = ""
    target_year = None
    cleaned_input = "" # Pour le logging d'erreur

    try:
        if isinstance(tool_input, str):
            cleaned_input = tool_input.strip()
            obs_marker = "\nObservation"
            if obs_marker in cleaned_input: # Gérer lva contamination
                cleaned_input = cleaned_input.split(obs_marker, 1)[0].strip()
            if cleaned_input.startswith("'") and cleaned_input.endswith("'"):
                cleaned_input = cleaned_input[1:-1]
        else: # Si ce n'est pas une chaîne, c'est une erreur d'appel
            return f"Erreur: tool_input doit être une chaîne, reçu {type(tool_input)}"

        try: # Essayer de parser comme JSON
            data = json.loads(cleaned_input)
            if isinstance(data, dict):
                query_str = data.get("query", "")
                target_year = data.get("target_year")
                print(f"   ℹ️ Input parsé comme JSON. Query: '{query_str}', Target Year: {target_year}")
            else: # JSON valide mais pas un dict
                query_str = cleaned_input
                print(f"   ℹ️ Input parsé comme JSON mais n'est pas un dict. Utilisation de l'input entier comme query: '{query_str}'")
        except (json.JSONDecodeError, TypeError): # Pas un JSON valide, traiter comme une simple chaîne
            query_str = cleaned_input
            year_match = re.search(r'\b(2019|2020|2021|2022|2023|2024|2025)\b', query_str)
            if year_match:
                target_year = year_match.group(1)
            print(f"   ℹ️ Input traité comme chaîne simple. Query: '{query_str}', Année détectée (si présente): {target_year}")

        if not query_str:
            return "Erreur: La requête (query) pour knowledge_base_retriever est vide après parsing."

        if 'loaded_vector_store' not in globals() or loaded_vector_store is None:
            return "Erreur: La base de données vectorielle 'loaded_vector_store' n'est pas chargée."

        k_retrieval = 7
        num_to_fetch = k_retrieval * 2 if target_year else k_retrieval

        all_results_with_scores = loaded_vector_store.similarity_search_with_score(query_str, k=num_to_fetch)

        if not all_results_with_scores:
            return "Aucune information pertinente trouvée pour cette requête."

        final_results_to_consider = []
        if target_year:
            print(f"   Filtrage des résultats pour l'année cible : {target_year} (parmi les {len(all_results_with_scores)} chunks récupérés)")
            for doc, score in all_results_with_scores:
                source_name = os.path.basename(doc.metadata.get('filename', doc.metadata.get('source', 'N/A')))
                if target_year in source_name:
                    final_results_to_consider.append((doc, score))

            if not final_results_to_consider and all_results_with_scores:
                 print(f"   ⚠️ Aucun résultat après filtrage strict pour {target_year}. Utilisation des {k_retrieval} meilleurs résultats globaux.")
                 final_results_to_consider = all_results_with_scores[:k_retrieval]
            elif final_results_to_consider: # Si on a des résultats filtrés, on les prend, jusqu'à k_retrieval
                final_results_to_consider = sorted(final_results_to_consider, key=lambda x: x[1])[:k_retrieval]
        else:
            final_results_to_consider = sorted(all_results_with_scores, key=lambda x: x[1])[:k_retrieval]

        if not final_results_to_consider:
             return "Aucune information pertinente trouvée après application des filtres ou globalement."

        contexts_for_llm = []
        sources_set = set()
        actual_scores_of_filtered_docs = []

        if DEBUG_SHOW_CHUNK_DETAILS:
            print(f"\n    Affichage des {len(final_results_to_consider)} chunks finaux pour la requête: '{query_str}' (année: {target_year})")
            print("\n=== DÉTAILS DES CHUNKS RÉCUPÉRÉS POUR ANALYSE ===\n")

        for i, (doc_obj, score) in enumerate(final_results_to_consider):
            source_name = os.path.basename(doc_obj.metadata.get('filename', doc_obj.metadata.get('source', 'N/A')))
            sources_set.add(source_name)
            contexts_for_llm.append(doc_obj.page_content)
            actual_scores_of_filtered_docs.append(score)

            if DEBUG_SHOW_CHUNK_DETAILS:
                # ... (affichage des chunks comme dans votre version)
                chunk_info = (
                    f"--- Chunk {i+1} (Source: {source_name}, Score de distance L2 : {score:.4f}) ---\n"
                    f"Contenu du Chunk:\n{doc_obj.page_content[:500]}...\n" # Limiter la longueur pour le log
                    f"Métadonnées du Chunk: {doc_obj.metadata}\n"
                )
                print(chunk_info)

        if DEBUG_SHOW_CHUNK_DETAILS:
            print("=== FIN DES DÉTAILS DES CHUNKS ===\n")

        final_context_for_llm = "\n\n---\n\n".join(contexts_for_llm)
        final_sources_str = ", ".join(sorted(list(sources_set)))
        scores_str = ", ".join([f"{s:.2f}" for s in actual_scores_of_filtered_docs])

        print(f"   ℹ️ Sources RAG finales fournies au LLM pour '{query_str}': {final_sources_str}")

        return (f"Informations trouvées (sources: {final_sources_str}; scores L2: {scores_str}):\n{final_context_for_llm}")

    except Exception as e:
        import traceback
        print(f"❌ Erreur détaillée dans knowledge_base_retriever pour l'input '{repr(tool_input)}': {traceback.format_exc()}")
        return f"Erreur lors de la recherche: {type(e).__name__} - {e}"

@tool
def get_stock_price_on_date(ticker: str, date_str: str) -> str:
    """
    Récupère le prix de clôture d'une action (ex: 'RNO.PA', '^FCHI') à une date spécifique.
    La date doit être au format 'YYYY-MM-DD'.
    """
    print(f"🛠️ Outil 'get_stock_price_on_date' appelé pour {ticker} le {date_str}")
    if 'stock_data' not in globals() or stock_data is None or stock_data.empty:
        return "Erreur: Les données boursières (`stock_data`) ne sont pas chargées ou sont vides."
    try:
        date_obj = pd.to_datetime(date_str)
        actual_date_in_index = stock_data.index.asof(date_obj)

        if pd.isna(actual_date_in_index):
             return f"Aucune donnée boursière trouvée avant ou à la date {date_str} pour {ticker}."

        price = stock_data.loc[actual_date_in_index, ('Close', ticker)]

        if pd.isna(price):
            return f"Aucun prix de clôture trouvé pour {ticker} à la date la plus proche ({actual_date_in_index.strftime('%Y-%m-%d')}) de {date_str} (donnée manquante)."
        return f"Le prix de clôture de {ticker} le {actual_date_in_index.strftime('%Y-%m-%d')} était de {price:.2f} EUR."
    except KeyError:
        available_tickers = []
        if isinstance(stock_data.columns, pd.MultiIndex):
            available_tickers = list(set(c[1] for c in stock_data.columns if c[0] == 'Close'))
        return f"Ticker '{ticker}' non trouvé parmi les colonnes 'Close'. Tickers disponibles: {available_tickers}"
    except Exception as e:
        return f"Erreur lors de la récupération du prix pour {ticker} le {date_str}: {e}"

@tool
def plot_vehicle_sales(sales_data_input: str) -> str:
    """
    Crée un graphique à barres illustrant le nombre de véhicules vendus par année.
    L'entrée doit être une chaîne de caractères représentant un dictionnaire Python,
    ex: "{'2020': 2950000, '2021': 2700000}".
    L'agent doit d'abord utiliser 'knowledge_base_retriever' pour obtenir ces chiffres.
    """
    print(f"🛠️ Outil 'plot_vehicle_sales' appelé.")
    print(f"   ➡️ Reçu sales_data_input (type {type(sales_data_input)}): {repr(sales_data_input)}")

    sales_data_str_cleaned = ""
    if isinstance(sales_data_input, str):
        raw_input_str = sales_data_input
    elif isinstance(sales_data_input, dict) and 'sales_data_str' in sales_data_input : # Si LLM passe un dict
        raw_input_str = sales_data_input['sales_data_str']
        if not isinstance(raw_input_str, str):
             return f"Erreur: 'sales_data_str' dans le dict d'entrée doit être une chaîne."
    else:
        return f"Erreur: L'entrée doit être une chaîne (représentant un dict) ou un dict avec la clé 'sales_data_str'. Reçu : {type(sales_data_input)}"

    obs_marker = "\nObservation"
    if obs_marker in raw_input_str:
        sales_data_str_cleaned = raw_input_str.split(obs_marker, 1)[0]
    else:
        sales_data_str_cleaned = raw_input_str
    sales_data_str_cleaned = sales_data_str_cleaned.strip()

    if sales_data_str_cleaned.startswith("'") and sales_data_str_cleaned.endswith("'"):
        sales_data_str_cleaned = sales_data_str_cleaned[1:-1]

    print(f"   ➡️ Chaîne à évaluer (sales_data_str_cleaned): {repr(sales_data_str_cleaned)}")

    try:
        sales_data = ast.literal_eval(sales_data_str_cleaned)
        if not isinstance(sales_data, dict):
            return "Erreur: Les données de ventes après évaluation ne forment pas un dictionnaire."

        years = sorted([str(y) for y in sales_data.keys()]) # Assurer que les années sont des chaînes pour l'axe X
        values = [float(sales_data[year]) for year in years]

        plt.figure(figsize=(10, 6))
        plt.bar(years, values, color='skyblue')
        plt.xlabel("Année")
        plt.ylabel("Nombre de véhicules vendus")
        plt.title("Ventes de véhicules Groupe Renault par année")
        plt.xticks(years)

        max_val = max(values) if values else 0
        for i, v_val in enumerate(values):
            display_val = f"{v_val/1000000:.2f}M" if v_val >= 100000 else str(int(v_val))
            offset = 0.02 * max_val if max_val > 0 else 0.02 * v_val
            plt.text(years[i], v_val + offset, display_val, ha='center', va='bottom') # Utiliser years[i] pour l'abscisse du texte

        plot_filename = "ventes_vehicules_par_an.png"
        plt.savefig(plot_filename)
        plt.close()
        return f"Graphique des ventes de véhicules sauvegardé sous '{plot_filename}'. Il illustre les ventes pour les années {', '.join(years)}."
    except SyntaxError as se:
        return f"Erreur de syntaxe lors de l'évaluation des données de ventes '{sales_data_str_cleaned}': {se}."
    except Exception as e:
        return f"Erreur inattendue lors de la création du graphique des ventes : {e}"

@tool
def plot_stock_vs_cac40_on_announcement_days(dates_input_raw: str) -> str:
    """
    Crée un graphique comparant le cours de l'action Renault (RNO.PA) et la performance
    du CAC40 les jours d'annonce de résultats depuis 2020.
    L'entrée doit être une chaîne de caractères représentant une liste de dates (ex: "['2021-02-19', '2022-02-18']")
    OU une chaîne JSON représentant un dictionnaire avec la clé "announcement_dates_str"
    (ex: '{"announcement_dates_str": "['2021-02-19', '2022-02-18']"}').
    """
    print(f"🛠️ Outil 'plot_stock_vs_cac40_on_announcement_days' appelé.")
    print(f"   ➡️ Reçu dates_input_raw (type {type(dates_input_raw)}): {repr(dates_input_raw)}")

    if not isinstance(dates_input_raw, str):
        return f"Erreur: L'entrée doit être une chaîne, mais reçu type {type(dates_input_raw)}"

    cleaned_input_str = dates_input_raw
    obs_marker = "\nObservation"
    if obs_marker in cleaned_input_str:
        cleaned_input_str = cleaned_input_str.split(obs_marker, 1)[0]
    cleaned_input_str = cleaned_input_str.strip()

    if cleaned_input_str.startswith("'") and cleaned_input_str.endswith("'"):
        cleaned_input_str = cleaned_input_str[1:-1]
    print(f"   ➡️ Chaîne nettoyée (cleaned_input_str) pour évaluation initiale: {repr(cleaned_input_str)}")

    actual_list_str = None
    announcement_dates_list = []

    try:
        # Essayer de déterminer si c'est une chaîne de dict ou une chaîne de liste
        if cleaned_input_str.startswith("{") and cleaned_input_str.endswith("}"):
            potential_dict = ast.literal_eval(cleaned_input_str)
            if isinstance(potential_dict, dict) and 'announcement_dates_str' in potential_dict:
                actual_list_str = potential_dict['announcement_dates_str']
                if not isinstance(actual_list_str, str):
                    return f"Erreur: 'announcement_dates_str' dans le JSON d'entrée doit être une chaîne de liste."
            else: # JSON valide mais pas le format attendu
                 return f"Erreur: JSON d'entrée non conforme, attendu clé 'announcement_dates_str'. Reçu: {repr(potential_dict)}"
        elif cleaned_input_str.startswith("[") and cleaned_input_str.endswith("]"):
            actual_list_str = cleaned_input_str # C'était déjà la chaîne de la liste
        else:
            return f"Erreur: L'entrée des dates n'est pas un format de liste ou de JSON de liste attendu. String nettoyée: {repr(cleaned_input_str)}"

        if actual_list_str is None:
            return f"Erreur: Impossible d'extraire la chaîne de la liste de dates à partir de: {repr(cleaned_input_str)}"

        # Nettoyer actual_list_str aussi
        actual_list_str = actual_list_str.strip()
        if actual_list_str.startswith("'") and actual_list_str.endswith("'"):
                actual_list_str = actual_list_str[1:-1]
        print(f"   ➡️ Chaîne de la liste de dates à évaluer (actual_list_str): {repr(actual_list_str)}")

        announcement_dates_list = ast.literal_eval(actual_list_str)

        if not isinstance(announcement_dates_list, list):
            return f"Erreur: Les dates parsées ne forment pas une liste: {repr(announcement_dates_list)}"

        announcement_dates = [pd.to_datetime(date_str) for date_str in announcement_dates_list] # Renommé 'date' en 'date_str'
        announcement_dates = sorted([d for d in announcement_dates if pd.notna(d) and d >= pd.to_datetime("2020-01-01")])


        if 'stock_data' not in globals() or stock_data is None or stock_data.empty:
            return "Erreur: Les données boursières (`stock_data`) ne sont pas chargées ou sont vides."
        if not announcement_dates:
            return "Aucune date d'annonce valide (post 2020-01-01 et convertible en date) fournie ou extraite."

        relevant_data = stock_data.reindex(pd.to_datetime(announcement_dates), method='ffill')
        if relevant_data.empty:
            return "Aucune donnée boursière trouvée pour les dates d'annonce spécifiées après reindexation."

        renault_prices = relevant_data.get(('Close', 'RNO.PA')) # Utiliser .get pour éviter KeyError si colonne absente
        cac40_prices = relevant_data.get(('Close', '^FCHI'))

        if renault_prices is None or cac40_prices is None or renault_prices.empty or cac40_prices.empty or \
           pd.isna(renault_prices.iloc[0]) or renault_prices.iloc[0] == 0 or \
           pd.isna(cac40_prices.iloc[0]) or cac40_prices.iloc[0] == 0:
            missing_info = []
            if renault_prices is None or renault_prices.empty or pd.isna(renault_prices.iloc[0]) or renault_prices.iloc[0] == 0: missing_info.append("RNO.PA")
            if cac40_prices is None or cac40_prices.empty or pd.isna(cac40_prices.iloc[0]) or cac40_prices.iloc[0] == 0: missing_info.append("CAC40")
            return f"Données boursières insuffisantes ou nulles pour {', '.join(missing_info)} aux dates d'annonce pour la normalisation."

        # S'assurer que les index sont bien des DatetimeIndex pour le plot
        renault_prices.index = pd.to_datetime(renault_prices.index)
        cac40_prices.index = pd.to_datetime(cac40_prices.index)

        renault_normalized = (renault_prices / renault_prices.iloc[0] * 100).ffill()
        cac40_normalized = (cac40_prices / cac40_prices.iloc[0] * 100).ffill()

        plt.figure(figsize=(12, 7))
        plt.plot(renault_normalized.index, renault_normalized, label='Renault (RNO.PA) Normalisé', marker='o')
        plt.plot(cac40_normalized.index, cac40_normalized, label='CAC40 (^FCHI) Normalisé', marker='x')
        plt.xlabel("Date d'annonce des résultats")
        plt.ylabel("Performance normalisée (Base 100)")
        plt.title("Performance Renault vs CAC40 les jours d'annonce de résultats")
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plot_filename = "renault_vs_cac40_annonces.png"
        plt.savefig(plot_filename)
        plt.close()
        return f"Graphique comparatif Renault vs CAC40 sauvegardé sous '{plot_filename}'. Données normalisées à 100 au premier jour d'annonce."
    except SyntaxError as se:
        err_actual_list_str = repr(actual_list_str) if actual_list_str is not None else 'Non défini (erreur avant extraction)'
        return f"Erreur de syntaxe lors de l'évaluation (cleaned_input_str: '{repr(cleaned_input_str)}', actual_list_str: '{err_actual_list_str}'): {se}."
    except Exception as e:
        return f"Erreur inattendue dans plot_stock_vs_cac40: {type(e).__name__} - {e}"

@tool
def analyze_sales_stock_correlation(json_input_str: str) -> str:
    """
    Analyse la corrélation entre les ventes annuelles de Renault et le cours de son action (RNO.PA)
    les jours d'annonce de résultats, depuis 2020.
    L'Action Input DOIT être un JSON string valide contenant les clés "sales_data_str" et "announcement_dates_str".
    Exemple: '{"sales_data_str": "{'2020': 2950000}", "announcement_dates_str": "['2021-02-19']"}'
    """
    print(f"🛠️ Outil 'analyze_sales_stock_correlation' appelé.")
    print(f"   ➡️ Reçu json_input_str (brut): {repr(json_input_str)}")

    if not isinstance(json_input_str, str):
        return f"Erreur: L'entrée doit être une chaîne JSON, reçu {type(json_input_str)}"

    cleaned_json_str = json_input_str
    obs_marker = "\nObservation"
    if obs_marker in cleaned_json_str:
        cleaned_json_str = cleaned_json_str.split(obs_marker, 1)[0]
    cleaned_json_str = cleaned_json_str.strip()

    if cleaned_json_str.startswith("'") and cleaned_json_str.endswith("'"):
        cleaned_json_str = cleaned_json_str[1:-1]

    print(f"   ➡️ Chaîne JSON nettoyée à parser: {repr(cleaned_json_str)}")

    sales_data_str_extracted = "Non extrait" # Pour le message d'erreur
    announcement_dates_str_extracted = "Non extrait" # Pour le message d'erreur

    try:
        data = {}
        try:
            data = json.loads(cleaned_json_str)
        except json.JSONDecodeError as je:
            print(f"   ⚠️ json.loads a échoué ({je}), tentative avec ast.literal_eval...")
            try:
                data = ast.literal_eval(cleaned_json_str)
            except Exception as ae:
                return f"Erreur de parsing final de '{cleaned_json_str}': json_err={je}, ast_err={ae}"

        if not isinstance(data, dict):
            return f"Erreur: L'entrée JSON parsée n'est pas un dictionnaire. Reçu: {repr(data)}"

        sales_data_str_extracted = data.get("sales_data_str")
        announcement_dates_str_extracted = data.get("announcement_dates_str")

        if sales_data_str_extracted is None or announcement_dates_str_extracted is None:
            return f"Erreur: JSON '{repr(cleaned_json_str)}' doit contenir 'sales_data_str' et 'announcement_dates_str'. Clés: {list(data.keys())}"
        if not isinstance(sales_data_str_extracted, str) or not isinstance(announcement_dates_str_extracted, str):
            return "Erreur: Les valeurs pour 'sales_data_str' et 'announcement_dates_str' dans le JSON doivent être des chaînes."

        print(f"      ➡️ sales_data_str extrait: {repr(sales_data_str_extracted)}")
        print(f"      ➡️ announcement_dates_str extrait: {repr(announcement_dates_str_extracted)}")

        sales_data = ast.literal_eval(sales_data_str_extracted)
        announcement_dates_list = ast.literal_eval(announcement_dates_str_extracted)

        if not isinstance(sales_data, dict) or not isinstance(announcement_dates_list, list):
             return "Erreur: sales_data_str ou announcement_dates_str n'ont pas été évalués en dict/list."

        announcement_dates_dt = [pd.to_datetime(d_str, errors='coerce') for d_str in announcement_dates_list]
        announcement_dates_dt = [d for d in announcement_dates_dt if pd.notna(d)] # Garder seulement les dates valides

        if 'stock_data' not in globals() or stock_data is None or stock_data.empty:
            return "Erreur: Les données boursières (`stock_data`) ne sont pas chargées ou sont vides."

        df_analysis = pd.DataFrame(index=pd.to_datetime(announcement_dates_dt)) # Assurer DatetimeIndex
        df_analysis = df_analysis[df_analysis.index.year >= 2020].sort_index()

        if df_analysis.empty:
            return "Aucune date d'annonce valide (convertible et >= 2020) pour l'analyse."

        df_analysis['stock_price'] = stock_data.reindex(df_analysis.index, method='ffill').get(('Close', 'RNO.PA'))

        def get_sales_for_announcement_year(ann_date):
            sales_year = str(ann_date.year - 1)
            return sales_data.get(sales_year)

        df_analysis['annual_sales'] = df_analysis.index.map(get_sales_for_announcement_year)

        print(f"      ➡️ df_analysis AVANT dropna: \n{df_analysis}") # DEBUG
        df_analysis.dropna(subset=['stock_price', 'annual_sales'], inplace=True)
        print(f"      ➡️ df_analysis APRES dropna: \n{df_analysis}") # DEBUG
        print(f"      ➡️ Longueur de df_analysis après dropna: {len(df_analysis)}") # DEBUG

        if len(df_analysis) < 2:
            return "Pas assez de données communes (ventes annuelles et prix de l'action après nettoyage) pour corrélation (besoin d'au moins 2 points)."

        correlation, p_value = pearsonr(df_analysis['annual_sales'].astype(float), df_analysis['stock_price'].astype(float))

        return (f"Analyse de corrélation (sur {len(df_analysis)} points):\n"
                f"- Coeff. Pearson : {correlation:.4f}\n- P-value : {p_value:.4f}\n"
                f"Interprétation: p-value < 0.05 suggère une corrélation significative.")
    except SyntaxError as se:
        return (f"Erreur de syntaxe lors de l'évaluation des chaînes d'entrée "
                f"(JSON global: '{cleaned_json_str}', sales_str: '{repr(sales_data_str_extracted)}', "
                f"dates_str: '{repr(announcement_dates_str_extracted)}'): {se}.")
    except Exception as e:
        import traceback
        print(f"❌ Erreur détaillée dans analyze_sales_stock_correlation: {traceback.format_exc()}")
        return f"Erreur inattendue dans l'analyse de corrélation : {type(e).__name__} - {e}"

print("✅ Outils définis.")

✅ Outils définis.


# 5. Exécution de l'Agent

In [143]:
# --- 5.1 Configuration du LLM ---
print("🧠 Configuration du LLM...")

# HUGGINGFACEHUB_API_TOKEN est supposé être chargé depuis la Partie 0 via .env ou défini autrement.
# Vérifions s'il est disponible et valide.
llm = None # Initialiser llm

# La variable HUGGINGFACEHUB_API_TOKEN est définie en Partie 0.
# HUGGINGFACEHUB_API_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

if not HUGGINGFACEHUB_API_TOKEN or HUGGINGFACEHUB_API_TOKEN == "hf_YOUR_TOKEN_HERE" or len(HUGGINGFACEHUB_API_TOKEN) < 10: # Ajout d'une vérification de base
    print("❌ ATTENTION : Le token API Hugging Face n'est pas configuré correctement dans l'environnement (via .env ou manuellement en Partie 0).")
    print("   L'agent LLM ne pourra pas fonctionner.")
else:
    print("✅ Token API Hugging Face trouvé et utilisé depuis l'environnement.")
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
    try:
        llm = HuggingFaceEndpoint(
            repo_id=repo_id,
            temperature=0.05,
            max_new_tokens=2048,
            huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN, # Utilise le token de l'environnement
            timeout=240
        )
        print(f"✅ LLM '{repo_id}' configuré via HuggingFaceEndpoint.")
    except Exception as e:
        print(f"❌ Erreur lors de la configuration du LLM HuggingFaceEndpoint: {e}")
        print("   Vérifiez votre token API et la disponibilité du modèle.")
        llm = None # S'assurer que llm est None en cas d'erreur

# --- 5.2 Définition de la liste des outils ---
# Les outils doivent être définis dans la Partie 4 et accessibles ici.
tools = []
if 'knowledge_base_retriever' in globals() and \
   'get_stock_price_on_date' in globals() and \
   'plot_vehicle_sales' in globals() and \
   'plot_stock_vs_cac40_on_announcement_days' in globals() and \
   'analyze_sales_stock_correlation' in globals():

    tools = [
        knowledge_base_retriever,
        get_stock_price_on_date,
        plot_vehicle_sales,
        plot_stock_vs_cac40_on_announcement_days,
        analyze_sales_stock_correlation
    ]
    print(f"🛠️ {len(tools)} outils prêts à être utilisés par l'agent.")
else:
    print("❌ ERREUR: Un ou plusieurs outils de la Partie 4 ne semblent pas définis. L'agent ne pourra pas les utiliser.")

# --- 5.3 Récupération du Prompt Template ---
prompt = None
try:
    prompt = hub.pull("hwchase17/react") # Prompt ReAct standard
    print("✅ Prompt ReAct chargé depuis Langchain Hub.")
except Exception as e:
    print(f"❌ Erreur lors du chargement du prompt depuis Langchain Hub: {e}. L'agent pourrait ne pas fonctionner comme prévu.")
    prompt = None

# --- 5.4 Création de l'Agent ---
agent = None
if llm and prompt and tools: # S'assurer que tous les composants sont là
    try:
        agent = create_react_agent(llm, tools, prompt)
        print("🤖 Agent créé.")
    except Exception as e:
        print(f"❌ Erreur lors de la création de l'agent: {e}")
        agent = None
else:
    print("⚠️ L'agent n'a pas pu être créé car des composants essentiels manquent (LLM, prompt, ou outils).")

# --- 5.5 Création de l'Exécuteur d'Agent ---
agent_executor = None
if agent and tools: # Agent et tools doivent exister
    try:
        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=SHOW_AGENT_THOUGHTS, # Contrôlé par la variable globale
            handle_parsing_errors=True,
            max_iterations=15 # Permettre plus d'itérations pour les tâches complexes
        )
        print("🏃 Exécuteur d'agent prêt.")
    except Exception as e:
        print(f"❌ Erreur lors de la création de l'exécuteur d'agent: {e}")
        agent_executor = None
else:
    if not agent: print("⚠️ Exécuteur d'agent non créé car l'agent manque.")
    if not tools: print("⚠️ Exécuteur d'agent non créé car la liste des outils est vide.")

agent_responses_list = [] # Initialiser la liste pour stocker les réponses

# --- 5.6 Test de l'Agent avec les Questions ---
if agent_executor:
    display(Markdown("--- \n# 🚀 Tests de l'Agent \n ---"))

    # Questions fidèles au brief PDF, en français
    questions = [
        "Résume le rapport du plan Renaulution quand il a été annoncé en 2021. Ta synthèse doit mettre en lumière la stratégie, les grandes étapes, et les principaux objectifs. Réponds en français.", # Q1
        "Identifie les membres du conseil d'administration en 2021. Réponds en français.", # Q2
        "Décris les changements au sein du conseil d'administration entre 2023 et 2024. Réponds en français.", # Q3
        "Quel était le nombre total de véhicules vendus par Renault en 2023 ? Réponds en français.", # Q4
        "Crée un graphique illustrant le nombre de véhicules vendus par Renault par année à partir de 2020. Assure-toi d'abord d'avoir les données de ventes pour les années 2020, 2021, 2022 et 2023. Réponds en français.", # Q5
        "Crée un graphique qui compare le cours de l'action Renault les jours d'annonce des résultats depuis 2020 avec la performance globale du CAC40 pendant la même période. Réponds en français.", # Q6
        "Analyse la corrélation entre les ventes annuelles de véhicules de Renault et le cours de son action, spécifiquement les jours d'annonce des résultats, depuis 2020. Réponds en français.", # Q7
        "Quels sont les indicateurs DPEF de Renault pour l'année 2023 ? Réponds en français.", # Q8
        "Résume les progrès du plan Renaulution depuis 2021. Réponds en français."  # Q9
    ]

    for i, question in enumerate(questions):
        display(Markdown(f"--- \n## ❓ Question {i+1}/{len(questions)}"))
        display(Markdown(f"**{question}**"))

        if not SHOW_AGENT_THOUGHTS:
            print(f"\n> Exécution de l'agent pour la question {i+1} (mode concis)...")
        else:
            print(f"\n> Exécution de l'agent pour la question {i+1} (mode verbeux)...")

        try:
            response = agent_executor.invoke({"input": question})
            final_answer = response.get("output", "Pas de sortie 'output' dans la réponse.")

            agent_responses_list.append({
                "id": f"Q{i+1}",
                "question_posee": question,
                "reponse_agent": final_answer
            })

            display(Markdown("--- \n### ✅ Réponse de l'Agent :"))
            display(Markdown(final_answer))

            if ".png" in final_answer.lower() or ".jpg" in final_answer.lower():
                try:
                    filename_match = re.search(r"(['\"])([^'\"]+\.(png|jpg))(['\"])", final_answer, re.IGNORECASE)
                    if filename_match:
                        filename = filename_match.group(2)
                        if os.path.exists(filename):
                            display(Markdown(f"🖼️ *Le fichier '{filename}' a été généré et est disponible.*"))
                            # display(Image(filename=filename)) # Optionnel: afficher l'image
                        else:
                            display(Markdown(f"🖼️ *L'agent mentionne '{filename}', mais le fichier n'est pas trouvé.*"))
                    else:
                        display(Markdown(f"🖼️ *Un fichier image semble avoir été mentionné.*"))
                except Exception as img_e:
                     display(Markdown(f"🖼️ *Erreur lors de la vérification du fichier image : {img_e}*"))
        except Exception as e:
            agent_responses_list.append({
                "id": f"Q{i+1}",
                "question_posee": question,
                "reponse_agent": f"ERREUR: {e}"
            })
            display(Markdown(f"### ❌ Erreur lors de l'exécution de l'agent :"))
            display(Markdown(f"```\n{e}\n```")) # Afficher l'erreur en format Markdown
        display(Markdown("---"))
else:
    # Ce message s'affichera si agent_executor n'a pas été initialisé à cause d'une erreur précédente
    display(Markdown("--- \n### ⚠️ L'exécuteur d'agent n'est pas initialisé. Vérifiez les logs des sections 5.1 à 5.5 pour des erreurs de configuration ou d'initialisation des composants (LLM, outils, prompt, agent).\n ---"))

🧠 Configuration du LLM...
✅ Token API Hugging Face trouvé et utilisé depuis l'environnement.
✅ LLM 'mistralai/Mixtral-8x7B-Instruct-v0.1' configuré via HuggingFaceEndpoint.
🛠️ 5 outils prêts à être utilisés par l'agent.
✅ Prompt ReAct chargé depuis Langchain Hub.
🤖 Agent créé.
🏃 Exécuteur d'agent prêt.


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


--- 
# 🚀 Tests de l'Agent 
 ---

--- 
## ❓ Question 1/9

**Résume le rapport du plan Renaulution quand il a été annoncé en 2021. Ta synthèse doit mettre en lumière la stratégie, les grandes étapes, et les principaux objectifs. Réponds en français.**

...

**Résume les progrès du plan Renaulution depuis 2021. Réponds en français.**


> Exécution de l'agent pour la question 9 (mode concis)...
🛠️ Outil 'knowledge_base_retriever' appelé avec tool_input: 'plan Renaulution objectifs"\nObservation'
   ℹ️ Input traité comme chaîne simple. Query: 'plan Renaulution objectifs"', Année détectée (si présente): None
   ℹ️ Sources RAG finales fournies au LLM pour 'plan Renaulution objectifs"': Renault_DEU_2020.pdf, Renault_DEU_2021.pdf, Renault_DEU_2022.pdf, Renault_DEU_2023.pdf, Renault_DEU_2024.pdf


--- 
### ✅ Réponse de l'Agent :

Le plan Renaulution est un plan stratégique de transformation de l'entreprise mis en place par M. Luca de Meo en janvier 2021. Il vise à transformer le Groupe pour bénéficier des opportunités de la mutation du secteur. La première phase, "Résurrection", a été achevée avec 3 ans d'avance et a porté ses fruits avec des résultats en 2022 au-dessus des objectifs initiaux et des attentes du marché. Le plan vise à accélérer l'efficacité des fonctions, renforcer l'efficacité et la performance de l'ingénierie et de la production, orienter la présence internationale du Groupe vers les marchés à fortes marges, et atteindre la neutralité carbone en Europe d'ici 2050.</s>

---

## 6. Comparaison et Évaluation des Réponses de l'Agent

In [144]:
target_answers = {
    "Q1": ("Le plan Renaulution, annoncé en 2021, vise une transformation de Renault d'une stratégie de volume à une de valeur. "
           "Il s'articule en 3 phases : Résurrection (redressement financier), Rénovation (renouvellement des gammes et focus sur segments rentables/VE), "
           "et Révolution (pivot vers tech, énergie, mobilité). Les marques sont repositionnées (Renault, Dacia, Alpine, Mobilize). "
           "Objectifs : réduction des coûts, amélioration de la rentabilité (ex: marge op. >3% en 2023, >5% en 2025), et lancement de ~14 modèles dont 7 VE d'ici 2025."),

    "Q2": ("Au 31 décembre 2021, le Conseil d'Administration de Renault comptait 17 membres. "
           "Parmi eux : Jean-Dominique Senard (Président), Catherine Barba, Frédéric Barrat, "
           "Miriem Bensalah-Chaqroun, Thomas Courbe, Marie-Annick Darmaillac, Bernard Delpit, Noël Desgrippes, "
           "Pierre Fleuriot, Richard Gentil, Frédéric Mazzella, Éric Personne, Yu Serizawa, Pascale Sourisse, "
           "Joji Tagawa, Annette Winkler, et Martin Vial (représentant de l'État français)."),

    "Q3": ("Les principaux changements au Conseil d'Administration durant l'année 2024 (visibles fin 2024 / début 2025) "
           "concernent les représentants des salariés : Frédéric Barrat et Eric Personne ont été remplacés par Sébastien Jacquet et Eric Vidal en novembre 2024. "
           "À noter également, Luca de Meo a été nommé administrateur en mai 2023."),

    "Q4": "En 2023, le Groupe Renault a vendu un total de 2 235 345 véhicules dans le monde.",

    "Q5": ("Le graphique des ventes du Groupe Renault devrait montrer approximativement : "
           "2020 : ~2,95M ; 2021 : ~2,70M ; 2022 : ~2,05M ; 2023 : ~2,24M véhicules. "
           "L'agent doit confirmer la génération du fichier graphique."),

    "Q6": ("Le graphique comparant RNO.PA et CAC40 aux dates d'annonce des résultats annuels "
           "(ex: pour résultats 2020: ~18/02/2021; 2021: ~17/02/2022; 2022: ~15/02/2023; 2023: ~14/02/2024) doit être généré. "
           "L'agent doit confirmer la génération du fichier."),

    "Q7": ("L'analyse devrait montrer une corrélation négative forte (environ -0.78) entre les ventes annuelles du Groupe Renault "
           "(2020-2023) et le cours de l'action RNO.PA aux jours d'annonce des résultats correspondants, "
           "suggérant que d'autres facteurs (rentabilité, stratégie) ont plus pesé sur le cours que les volumes durant cette période."),

    "Q8": ("Pour 2023, les indicateurs DPEF clés de Renault Group incluent : "
           "Émissions CO2 usines (scope 1+2) : ~612 kt ; "
           "Émissions CO2 véhicules vendus (Europe) : ~99,1 g/km ; "
           "Part d'électricité renouvelable (sites indus.) : ~37% ; "
           "Taux de fréquence des accidents du travail (TF1) : ~1.66 ; "
           "Part de femmes dans l'effectif total : ~25,1%."),

    "Q9": ("Depuis 2021, le plan Renaulution a permis un redressement significatif : "
           "Phase Résurrection (2021-2022) achevée en avance (marge op. 5,6% en 2022, FCF 2,1 Mds€). "
           "2023 (Rénovation/Révolution) : marge op. record 7,9%, FCF 3 Mds€, réorganisation en 5 business units. "
           "Focus sur valeur, rentabilité, électrification et nouvelles mobilités.")
}

# --- 6.2 Affichage de la Comparaison et Évaluation Qualitative ---

if 'agent_responses_list' in globals() and agent_responses_list:
    display(Markdown("--- \n# 📊 Comparaison et Évaluation des Réponses de l'Agent 📊 \n ---"))

    for i, response_item in enumerate(agent_responses_list):
        q_id = response_item["id"]
        question_posee_par_agent = response_item["question_posee"] # C'est la question de votre liste
        reponse_agent_val = response_item["reponse_agent"]

        # Pour la comparaison, nous utilisons la question originale du brief (stockée dans la liste `questions`)
        # Assurez-vous que `questions` est accessible ici ou recréez la liste.
        # Si vous avez utilisé un autre nom de liste en Partie 5, mettez-le à jour ici.
        try:
            question_originale_du_brief = questions[i] # questions de la Partie 5
        except NameError: # Si la liste n'est pas définie dans cette portée
            question_originale_du_brief = question_posee_par_agent # Fallback
        except IndexError:
             question_originale_du_brief = f"Question originale non trouvée pour Q{i+1}"


        target_reponse_val = target_answers.get(q_id, "Réponse attendue simplifiée non définie pour cette question.")

        display(Markdown(f"## ❓ {q_id} : {question_originale_du_brief}")) # Affiche la question du brief

        display(Markdown("### ✅ Réponse de l'Agent:"))
        display(Markdown(f"```text\n{reponse_agent_val}\n```"))

        display(Markdown("### 🎯 Réponse Attendue :"))
        display(Markdown(f"```text\n{target_reponse_val}\n```"))

        # --- Section d'Évaluation Qualitative (Exemple) ---
        evaluation_commentaire = "Évaluation qualitative :"
        # Ici, vous (ou moi, dans ce rôle) pourriez ajouter une logique d'évaluation
        # Pour l'instant, je vais mettre un commentaire générique.
        # Dans un vrai scénario, on comparerait plus en détail.

        if "ERREUR LORS DE L'EXÉCUTION" in reponse_agent_val:
            evaluation_commentaire += "\n- L'agent a rencontré une erreur."
        elif "graphique" in question_originale_du_brief.lower():
            if ".png" in reponse_agent_val.lower() or "graphique sauvegardé" in reponse_agent_val.lower() :
                 evaluation_commentaire += "\n- L'agent semble avoir correctement généré ou tenté de générer le graphique."
            else:
                 evaluation_commentaire += "\n- L'agent n'a pas confirmé la génération du graphique."
        elif q_id == "Q7" and "corrélation" in question_originale_du_brief.lower():
            if "corrélation" in reponse_agent_val.lower() and ("Pearson" in reponse_agent_val or "P-value" in reponse_agent_val):
                evaluation_commentaire += "\n- L'agent a fourni une analyse de corrélation."
            elif "Pas assez de données" in reponse_agent_val or "Aucune date" in reponse_agent_val :
                evaluation_commentaire += "\n- L'agent indique un manque de données pour la corrélation (à vérifier par rapport aux logs des outils)."
            else:
                evaluation_commentaire += "\n- La réponse pour la corrélation semble incomplète ou manquante."

        # Vous pouvez ajouter d'autres logiques d'évaluation ici pour chaque question.
        # Par exemple, vérifier la présence de mots-clés attendus.
        # Pour Q4, on pourrait essayer d'extraire le chiffre et de le comparer.
        if q_id == "Q4":
            match_agent = re.search(r"(\d{1,3}(?:\s?\d{3})*|\d+)", reponse_agent_val.replace(" ", ""))
            match_target = re.search(r"(\d{1,3}(?:\s?\d{3})*|\d+)", target_reponse_val.replace(" ", ""))
            if match_agent and match_target:
                if match_agent.group(1) == match_target.group(1):
                    evaluation_commentaire += f"\n- Ventes 2023 : Chiffre {match_agent.group(1)} correctement identifié."
                else:
                    evaluation_commentaire += f"\n- Ventes 2023 : Chiffre de l'agent ({match_agent.group(1)}) diffère de l'attendu ({match_target.group(1)})."
            else:
                evaluation_commentaire += f"\n- Ventes 2023 : Impossible d'extraire/comparer les chiffres de ventes."

        display(Markdown(f"**📝 {evaluation_commentaire}**"))
        display(Markdown("---"))

else:
    if not ('agent_responses_for_evaluation' in globals() and agent_responses_list):
        print("⚠️ Aucune réponse de l'agent n'a été stockée depuis la Partie 5.")
    if not target_answers: # Ne devrait pas arriver si le dict est défini
        print("⚠️ Les réponses attendues simplifiées ne sont pas définies.")

--- 
# 📊 Comparaison et Évaluation des Réponses de l'Agent 📊 
 ---

## ❓ Q1 : Résume le rapport du plan Renaulution quand il a été annoncé en 2021. Ta synthèse doit mettre en lumière la stratégie, les grandes étapes, et les principaux objectifs. Réponds en français.

### ✅ Réponse de l'Agent:

```text
Le plan Renaulution, annoncé en 2021, est une stratégie de transformation de l'entreprise en trois étapes : la résurrection (2020-2023), la rénovation (2022-2025), et la révolution (à partir de 2025). La résurrection vise à réduire les coûts et améliorer les marges, tandis que la rénovation introduira une gamme entièrement nouvelle et électrifiée, se recentrant sur les véritables gisements de bénéfices. La révolution, à partir de 2025, vise à se recentrer sur la chaîne de valeur de la nouvelle mobilité, en répondant aux nouvelles attentes des clients. Le plan prévoit également une réduction des coûts fixes et variables, avec un objectif de 3 milliards d'euros d'ici 2025. Les résultats de l'année 2021 ont validé la première étape du redressement.</s>
```

### 🎯 Réponse Attendue :

...

```text
Le plan Renaulution est un plan stratégique de transformation de l'entreprise mis en place par M. Luca de Meo en janvier 2021. Il vise à transformer le Groupe pour bénéficier des opportunités de la mutation du secteur. La première phase, "Résurrection", a été achevée avec 3 ans d'avance et a porté ses fruits avec des résultats en 2022 au-dessus des objectifs initiaux et des attentes du marché. Le plan vise à accélérer l'efficacité des fonctions, renforcer l'efficacité et la performance de l'ingénierie et de la production, orienter la présence internationale du Groupe vers les marchés à fortes marges, et atteindre la neutralité carbone en Europe d'ici 2050.</s>
```

### 🎯 Réponse Attendue :

```text
Depuis 2021, le plan Renaulution a permis un redressement significatif : Phase Résurrection (2021-2022) achevée en avance (marge op. 5,6% en 2022, FCF 2,1 Mds€). 2023 (Rénovation/Révolution) : marge op. record 7,9%, FCF 3 Mds€, réorganisation en 5 business units. Focus sur valeur, rentabilité, électrification et nouvelles mobilités.
```

**📝 Évaluation qualitative :**

---